In [1]:
import fasttext
import time

# 詞向量模型學習
## 使用supervised

## Skipgram model

In [6]:
%%time

# Skipgram model

# 執行完會輸出一個 model.bin 以及 model.vec 在資料夾底下

# 這個是處理過得文字檔案  有label
# skip_model = fasttext.skipgram('hotel_label_clean_louis1.txt', 'skip_model')  

#這個是只有文字  還未處理過  沒有label
#  skip_model = fasttext.skipgram('hotel_label_clean_onlycontent1.txt', 'skip_model') 

# 這個是處理過得文字檔案  有label 但是只有postive跟negative  沒有編號1跟2
skip_model = fasttext.skipgram('./data/hotel_label_onlyPN_train.txt', './model/skip_model'
                              ,dim = 100, epoch = 7, lr = 0.4  )

CPU times: user 17.2 s, sys: 2.93 s, total: 20.1 s
Wall time: 33.7 s


In [7]:
# 全部有幾個字詞
len(skip_model.words)

3643

In [8]:
print('dim =',skip_model.dim ,',' ,
      'bucket=', skip_model.bucket )

dim = 100 , bucket= 2000000


## CBOW model

In [6]:
%%time

# CBOW model

# 執行完會輸出一個 model.bin 以及 model.vec 在資料夾底下

#cbow_model = fasttext.cbow('hotel_label_clean_louis1.txt', 'model')  #這個是處理過得文字檔案  有label

#這個是只有文字  還未處理過  沒有label
#cbow_model = fasttext.cbow('hotel_label_clean_onlycontent1.txt', 'cbow_model')

# 這個是處理過得文字檔案  有label 但是只有postive跟negative  沒有編號1跟2
cbow_model = fasttext.cbow('./data/hotel_label_clean_onlyPN.txt', './model/cbow_model')  


CPU times: user 12.4 s, sys: 1.14 s, total: 13.6 s
Wall time: 19.9 s


In [7]:
# 全部有幾個字詞
len(cbow_model.words)

3870

# 切割資料集
## 目前是用20%測試

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import cross_val_score
import pandas as pd

In [45]:
# 記得要用 header = none 因為沒有標題
data = pd.read_csv('./data/hotel_label_clean_onlyPN.csv',header=None)

In [46]:
train, test = train_test_split(data, test_size = 0.2  )

In [47]:
# 原本資料共有
data.shape

(24033, 2)

In [48]:
# 訓練資料集共有
train.shape

(19226, 2)

In [49]:
train.head(5)

,0,1
13212,__label__POSITIVE,the staff assisted us in securing taxis ticke...
16112,__label__POSITIVE,the owners know almost everything there is to ...
13239,__label__POSITIVE,having said that- we would highly recommend st...
8477,__label__POSITIVE,and the cafes are fine for a croissant or a ro...
11100,__label__POSITIVE,but for the rest of us with a couple hunderd e...


In [50]:
# 先存成txt  看要轉csv 或 txt都可以
train.to_csv('./data/hotel_label_onlyPN_train.txt', index=None, header=None)

In [51]:
# 測試資料集共有
test.shape

(4807, 2)

In [52]:
test.head(5)

,0,1
3705,__label__NEGATIVE,i can say that this was possibly the worst hot...
5859,__label__POSITIVE,good location nice decor but very small room ...
3170,__label__POSITIVE,a very nice start to our weekend in amsterdam .
22208,__label__POSITIVE,swissotel sydney is in a great location
21036,__label__POSITIVE,we were give a rate which was very reasonable ...


In [53]:
# 先存成txt  看要轉csv 或 txt都可以
test.to_csv('./data/hotel_label_onlyPN_test.txt', index=None, header=None)

# 文章分類 - 分類模型(訓練分類器)

### supervised 一定要吃有label的文件

In [54]:
import fasttext
import time

In [55]:
%%time

# 執行完會輸出一個 model.bin  在資料夾底下

# 參數
label_prefix='__label__'

dim=10
epoch=10
lr=0.9
ws=15
neg=3
minn=1


#thread=12
#loss='ns'

# 這邊覆蓋掉了前面的沒有使用supervised模型
# 這個是處理過得文字檔案  有label
skip_classifier = fasttext.supervised('./data/hotel_label_onlyPN_train.txt', './model/skip_model'
                    ,dim = dim, epoch = epoch, lr = lr, ws= ws, neg= neg, minn= minn
#                     , thread= thread , loss= loss
                    ,label_prefix= label_prefix)  


CPU times: user 945 ms, sys: 67.2 ms, total: 1.01 s
Wall time: 1.81 s


# 評估準確率

In [56]:
# 預測這個文字檔試試看  需要有label的文件
skip_result = skip_classifier.test('./data/hotel_label_onlyPN_test.txt')  #這個是處理過得文字檔案  有label

In [57]:
print ('SkipgramModel precision =',skip_result.precision)

SkipgramModel precision = 0.8493863116288746


In [15]:
# dim 字詞維度
# epoch 訓練次數
# minN-gram  最小 N-gram
# maxN-gram  最大 N-gram
# word_ngrams
# min_count

print (
'dim =' ,skip_classifier.dim,'\n'
'epoch =' ,skip_classifier.epoch,'\n'
'minn =',skip_classifier.minn,'\n'
'maxn =',skip_classifier.maxn,'\n'
'word_ngrams =' ,skip_classifier.word_ngrams,'\n'
'min_count =' , skip_classifier.min_count,'\n'
    )

dim = 10 
epoch = 10 
minn = 1 
maxn = 0 
word_ngrams = 1 
min_count = 1 



# 訓練模型 - 參數說明

In [5]:
# 目前試過的組合 
# dim  100 ,200 ,300  效果差不多  85%
# epoch  5 =85.399 ,  6 =85.482 , 7 =85.607 , 10 =85.440
# lr  0.2 =85.274 , 0.3 =85.357 , 0.4 =85.648 ,  0.5 =85.149
# ws  10 =85.357 , 11 =85.440 , 12=85.524 , 13 =85.856 , 14 =85.191
# loss softmax =85.482 , ns =85.607 , hs =85.191
# neg 1 =85.856 ,2 =85.732 , 3 =85.357, 4 =85.856 , 5 =85.524 , 6 =85.316 
# minn 1 =85.274 , 2 =85.565 , 3 =85.066
# thread 10 =85.024 , 11 = 85.690 ,12 =85.482 , 13=85.565 , 14 =85.191 , 15 = 85.108

# 評估模型準確率 

## skipgram

In [24]:
# 預測這個文字檔試試看  需要有label的文件
skip_result = skip_classifier.test('./data/hotel_label_onlyPN_test.txt')  #這個是處理過得文字檔案  有label

In [25]:
print ('SkipgramModel precision =',skip_result.precision)

SkipgramModel precision = 0.8548252911813644


In [26]:
print ('SkipgramModel recall =',skip_result.recall) 

SkipgramModel recall = 0.8548252911813644


# 預測一句話

## skip_classifier

In [14]:
# 看是正向還是負向
skip_classifier.predict(["I like it"])

[['NEGATIVE']]

In [15]:
# 看是那一個面向的機率有多少
skip_classifier.predict_proba(["I don't like it"])

[[('NEGATIVE', 0.800781)]]

In [16]:
# 看是兩個面向的機率有多少
skip_classifier.predict_proba(["I don't like it"],2)

[[('NEGATIVE', 0.800781), ('POSITIVE', 0.197266)]]

In [ ]:
# 底下想要測試隨便一句話  不是飯店留言 直接丟進去跑出來結果會如何  但不太準確

In [23]:
texts = [ 'example very longtext so I think that is not suitable'
        ,'Washington Posts Attack Article on Google Duplex made me throw Alexa in the trash!'
        ,'good nice well done'] 

In [24]:
labels = skip_classifier.predict(texts)

In [25]:
print (labels)

[['NEGATIVE'], ['POSITIVE'], ['POSITIVE']]


In [26]:
# 測試飯店留言看看?
# test1 = 正向 , test2 = 負向
text1 = ['./data/test1_labal','./data/test2_labal']

In [27]:
label1 = skip_classifier.predict(text1)

In [30]:
# 看得出來1準確  2不準確
print (label1)

[['POSITIVE'], ['POSITIVE']]
